<a href="https://colab.research.google.com/github/jasonheesanglee/LLM_Study/blob/main/SQLAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 사용 사례

엔터프라이즈 데이터는 종종 SQL 데이터베이스에 저장됩니다.

LLM을 사용하면 자연어를 사용하여 SQL 데이터베이스와 상호 작용할 수 있습니다.

LangChain은 자연어 프롬프트를 기반으로 SQL 쿼리를 작성하고 실행할 수 있는 SQL 체인 및 에이전트를 제공합니다.

이들은 SQLAlchemy에서 지원하는 모든 SQL 언어(예: MySQL, PostgreSQL, Oracle SQL, Databricks, SQLite)와 호환됩니다.

다음과 같은 사용 사례를 지원합니다:

- 자연어 질문을 기반으로 실행할 쿼리 생성
- 데이터베이스 데이터를 기반으로 질문에 답변할 수 있는 챗봇 만들기
- 사용자가 분석하고자 하는 인사이트를 기반으로 사용자 지정 대시보드 구축

## 개요

LangChain은 SQL 데이터베이스와 상호 작용할 수 있는 도구를 제공합니다:

1. 자연어 사용자 질문을 기반으로 'SQL 쿼리 작성'
2. 쿼리 생성 및 실행을 위해 체인을 사용하여 'SQL 데이터베이스 쿼리'를 수행합니다.
3. 강력하고 유연한 쿼리를 위해 에이전트를 사용하여 'SQL 데이터베이스와 상호작용하기'

## 빠른 시작

먼저 필요한 패키지를 가져오고 환경 변수를 설정합니다:

In [ ]:
! pip install -qqq langchain langchain-experimental openai tiktoken cohere

import os
import openai
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OpenAI_API')

아래 예는 Chinook DB와의 SQLite 연결을 수행합니다.

* [이 파일](https://www.sqlitetutorial.net/sqlite-sample-database/)을 디렉터리에 `Chinook_Sqlite.sql`로 저장합니다.
* `sqlite3 Chinook.db`를 실행합니다.
* `.read Chinook_Sqlite.sql` 실행
* 테스트 `SELECT * FROM Artist LIMIT 10;`

이제 `Chinhook.db`가 디렉토리에 있습니다.

SQL 쿼리를 생성하고 실행하기 위해 `SQLDatabaseChain`을 생성해 보겠습니다.

In [29]:
import shutil
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [33]:
shutil.copyfile(src='/content/drive/MyDrive/Colab Notebooks/LangChain/chinook.db', dst='/content/chinook.db')

'/content/chinook.db'

In [34]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

import logging
logging.disable(logging.CRITICAL)
logging.disable(logging.WARNING)
logging.disable(logging.INFO)
logging.disable(logging.DEBUG)


from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase.from_uri("sqlite:///chinook.db")
llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [35]:
db_chain.run("몇명의 직원이 있어?")



> Entering new SQLDatabaseChain chain...
몇명의 직원이 있어?
SQLQuery:SELECT COUNT(*) FROM employees
SQLResult: [(8,)]
Answer:8
> Finished chain.


'8'

## Case 1: Text-to-SQL query

In [11]:
from langchain.chains import create_sql_query_chain
from langchain.chat_models import ChatOpenAI

SQL 쿼리를 작성할 체인을 만들어 보겠습니다:

In [12]:
chain = create_sql_query_chain(ChatOpenAI(temperature=0), db)
response = chain.invoke({"question": "How many employees are there"})
print(response)

SELECT COUNT(*) FROM employees


사용자 질문을 기반으로 SQL 쿼리를 작성하고 나면 쿼리를 실행할 수 있습니다:

In [13]:
db.run(response)

'[(8,)]'

## Case 2: Text-to-SQL query and execution

We can use `SQLDatabaseChain` from `langchain_experimental` to create and run SQL queries.

In [9]:
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain

llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [10]:
db_chain.run("직원이 몇명이나 있어?")



> Entering new SQLDatabaseChain chain...
직원이 몇명이나 있어?
SQLQuery:SELECT COUNT(*) FROM employees
SQLResult: [(8,)]
Answer:8
> Finished chain.


'8'

보시다시피 이전 사례와 동일한 결과를 얻을 수 있습니다.

여기서 체인은 **쿼리 실행도 처리**하고 사용자 질문과 쿼리 결과를 기반으로 최종 답변을 제공합니다.

이 방식은 'SQL 인젝션'에 취약하기 때문에 사용 시 **주의**해야 합니다:

* 체인이 LLM에 의해 생성되고 검증되지 않은 쿼리를 실행하고 있습니다.
* 예: 레코드가 의도치 않게 생성, 수정 또는 삭제될 수 있음_.

이것이 바로 `SQLDatabaseChain`이 `랭체인_실험` 안에 있는 이유입니다.

## 사례 3: SQL 에이전트

LangChain에는 `SQLDatabaseChain`보다 SQL 데이터베이스와 상호 작용하는 더 유연한 방법을 제공하는 SQL 에이전트가 있습니다.

SQL 에이전트 사용의 주요 장점은 다음과 같습니다:

- 데이터베이스의 스키마뿐만 아니라 데이터베이스의 콘텐츠(예: 특정 테이블 설명)를 기반으로 질문에 답변할 수 있습니다.
- 생성된 쿼리를 실행하고 트레이스백을 포착하여 올바르게 다시 생성함으로써 오류로부터 복구할 수 있습니다.

에이전트를 초기화하기 위해 `create_sql_agent` 함수를 사용합니다.

이 에이전트에는 다음과 같은 도구가 포함된 `SQLDatabaseToolkit`이 포함되어 있습니다:

* 쿼리 생성 및 실행
* 쿼리 구문 확인
* 테이블 설명 검색
* ... 등

In [14]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

# from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType

db = SQLDatabase.from_uri("sqlite:///chinook.db")

agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0)),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

### Agent task example #1 - Running queries


In [15]:
agent_executor.run(
    "국가별 총 매출을 나열합니다. 어느 국가의 고객이 가장 많이 지출했나요?"
)



> Entering new AgentExecutor chain...
 We need to find the total sales for each country and then determine which country has the highest total sales.
Action: sql_db_query
Action Input: SELECT country, SUM(sales) AS total_sales FROM customers GROUP BY country ORDER BY total_sales DESC LIMIT 1Error: (sqlite3.OperationalError) no such column: sales
[SQL: SELECT country, SUM(sales) AS total_sales FROM customers GROUP BY country ORDER BY total_sales DESC LIMIT 1]
(Background on this error at: https://sqlalche.me/e/20/e3q8)We need to check the table schema to see if there is a column named 'sales'.
Action: sql_db_schema
Action Input: customers
CREATE TABLE customers (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60)

'The USA has the highest total sales.'

### Agent task example #2 - Describing a Table

In [16]:
agent_executor.run("playlisttrack 테이블에 대해서 설명해줄래?")



> Entering new AgentExecutor chain...
 I should use sql_db_schema to get the schema and sample rows for the playlisttrack table.
Action: sql_db_schema
Action Input: playlisttrackError: table_names {'playlisttrack'} not found in databaseI should use sql_db_list_tables to get a list of tables in the database and check if playlisttrack is included.
Action: sql_db_list_tables
Action Input: ""albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracksI should use sql_db_schema again with the correct spelling of playlist_track.
Action: sql_db_schema
Action Input: playlist_track
CREATE TABLE playlist_track (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES tracks ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES playlists ("PlaylistId")
)

/*
3 rows from playlist_track table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/I now know the final answ

'The playlisttrack table has two columns, PlaylistId and TrackId, and is used to store the relationship between playlists and tracks. It has a composite primary key consisting of both columns and foreign key constraints referencing the playlists and tracks tables. Sample rows from the table show the relationship between a playlist with id 1 and three tracks with ids 3402, 3389, and 3390.'

### SQL 툴킷 확장하기

기본 제공되는 SQL 툴킷에는 데이터베이스 작업을 시작하는 데 필요한 도구가 포함되어 있지만, 에이전트의 기능을 확장하는 데 몇 가지 추가 도구가 유용할 수 있는 경우가 종종 있습니다. 이는 솔루션의 전반적인 성능을 개선하기 위해 솔루션에서 **도메인별 지식**을 사용하려고 할 때 특히 유용합니다.

몇 가지 예는 다음과 같습니다:

- Dynamic Few shot 예시 포함
- 열 필터로 사용할 고유명사의 철자 오류 찾기

이러한 특정 사용 사례를 처리하는 별도의 도구를 만들어 표준 SQL 도구 키트에 보완용으로 포함할 수 있습니다. 이 두 가지 사용자 정의 도구를 포함하는 방법을 살펴보겠습니다.

#### Dynamic Few shot 예제 포함

Dynamic Few shot 예제를 포함하려면 사용자의 질문과 의미적으로 유사한 예제를 검색하기 위해 벡터 데이터베이스를 처리하는 사용자 지정 **검색 도구**가 필요합니다.

몇 가지 예제가 포함된 사전을 만드는 것부터 시작하겠습니다:

In [17]:
few_shots = {
    "List all artists.": "SELECT * FROM artists;",
    "Find all albums for the artist 'AC/DC'.": "SELECT * FROM albums WHERE ArtistId = (SELECT ArtistId FROM artists WHERE Name = 'AC/DC');",
    "List all tracks in the 'Rock' genre.": "SELECT * FROM tracks WHERE GenreId = (SELECT GenreId FROM genres WHERE Name = 'Rock');",
    "Find the total duration of all tracks.": "SELECT SUM(Milliseconds) FROM tracks;",
    "List all customers from Canada.": "SELECT * FROM customers WHERE Country = 'Canada';",
    "How many tracks are there in the album with ID 5?": "SELECT COUNT(*) FROM tracks WHERE AlbumId = 5;",
    "Find the total number of invoices.": "SELECT COUNT(*) FROM invoices;",
    "List all tracks that are longer than 5 minutes.": "SELECT * FROM tracks WHERE Milliseconds > 300000;",
    "Who are the top 5 customers by total purchase?": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM invoices GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;",
    "Which albums are from the year 2000?": "SELECT * FROM albums WHERE strftime('%Y', ReleaseDate) = '2000';",
    "How many employees are there": 'SELECT COUNT(*) FROM "employee"',
}

그런 다음 질문 목록을 사용하여 검색기를 생성하고 대상 SQL 쿼리를 메타데이터로 할당할 수 있습니다:

In [18]:
!pip install -qqq tiktoken faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 56.6 MB/s eta 0:00:00


In [19]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings()

few_shot_docs = [
    Document(page_content=question, metadata={"sql_query": few_shots[question]})
    for question in few_shots.keys()
]
vector_db = FAISS.from_documents(few_shot_docs, embeddings)
retriever = vector_db.as_retriever()

이제 고유한 사용자 지정 도구를 만들어 'create_sql_agent' 함수에 새 도구로 추가할 수 있습니다:

In [20]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool_description = """
이 도구는 유사한 예시를 이해하여 사용자 질문에 적용하는 데 도움이 됩니다.
이 도구에 입력하는 내용은 사용자 질문이어야 합니다.
"""

retriever_tool = create_retriever_tool(
    retriever, name="sql_get_similar_examples", description=tool_description
)
custom_tool_list = [retriever_tool]

이제 사용 사례를 고려하여 표준 SQL 에이전트 접미사를 조정하여 에이전트를 만들 수 있습니다. 이를 처리하는 가장 간단한 방법은 도구 설명에 포함시키는 것이지만, 이것만으로는 충분하지 않은 경우가 많으므로 생성자의 '접미사' 인수를 사용하여 에이전트 프롬프트에서 이를 지정해야 합니다.

In [21]:
from langchain.agents import AgentType, create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///chinook.db")
llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

custom_suffix = """
먼저 제가 알고 있는 비슷한 예제를 가져와야 합니다.
예제가 쿼리를 구성하기에 충분하다면 쿼리를 작성할 수 있습니다.
그렇지 않으면 데이터베이스의 테이블을 살펴보고 쿼리할 수 있는 항목을 확인할 수 있습니다.
그런 다음 가장 관련성이 높은 테이블의 스키마를 쿼리해야 합니다.
"""

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    extra_tools=custom_tool_list,
    suffix=custom_suffix,
)

Let's try it out:

In [22]:
agent.run("How many employees do we have?")



> Entering new AgentExecutor chain...

Invoking: `sql_get_similar_examples` with `{'query': 'How many employees do we have?'}`


How many employees are there

Find the total number of invoices.

Who are the top 5 customers by total purchase?

List all customers from Canada.
Invoking: `sql_db_list_tables` with ``


albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracks
Invoking: `sql_db_schema` with `{'table_names': 'employees'}`



CREATE TABLE employees (
	"EmployeeId" INTEGER NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"FirstName" NVARCHAR(20) NOT NULL, 
	"Title" NVARCHAR(30), 
	"ReportsTo" INTEGER, 
	"BirthDate" DATETIME, 
	"HireDate" DATETIME, 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60), 
	PRIMARY KEY ("EmployeeId"), 
	FOREIGN KEY("ReportsTo") REFERENCES employee

'We have a total of 8 employees.'

#### 고유명사의 맞춤법 오류 찾기 및 수정하기

주소, 노래 이름 또는 아티스트와 같은 고유명사가 포함된 열을 필터링하려면 먼저 철자를 다시 확인하여 데이터를 올바르게 필터링해야 합니다.

데이터베이스에 존재하는 모든 고유 고유명사를 사용하여 벡터 저장소를 생성하면 됩니다. 그런 다음 사용자가 질문에 고유 명사를 포함할 때마다 상담원이 해당 벡터 저장소를 쿼리하여 해당 단어의 올바른 철자를 찾도록 할 수 있습니다. 이러한 방식으로 에이전트는 대상 쿼리를 작성하기 전에 사용자가 어떤 엔티티를 참조하는지 이해할 수 있습니다.

메타데이터 없이 고유명사를 임베드한 다음 철자가 틀린 사용자 질문과 가장 유사한 것을 쿼리하는 방식으로 몇 가지 샷과 유사한 접근 방식을 따라 해 보겠습니다.

먼저 원하는 각 엔티티에 대한 고유 값이 필요하며, 이를 위해 결과를 요소 목록으로 파싱하는 함수를 정의합니다:

In [23]:
print(db.table_info)


CREATE TABLE albums (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES artists ("ArtistId")
)

/*
3 rows from albums table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE artists (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from artists table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE customers (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES employe

In [24]:
import ast
import re


def run_query_save_results(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return res


artists = run_query_save_results(db, "SELECT name FROM artists")
albums = run_query_save_results(db, "SELECT title FROM albums")

이제 사용자 지정 **리트리버 도구**와 최종 에이전트 생성을 진행할 수 있습니다:

In [25]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

texts = artists + albums

embeddings = OpenAIEmbeddings()
vector_db = FAISS.from_texts(texts, embeddings)
retriever = vector_db.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    name="name_search",
    description="이름, 성 주소 등 데이터가 실제로 어떻게 쓰여졌는지 알아내는 데 사용합니다.",
)

custom_tool_list = [retriever_tool]

In [26]:
from langchain.agents import AgentType, create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SQLDatabase

# db = SQLDatabase.from_uri("sqlite:///Chinook.db")
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

custom_suffix = """
사용자가 고유명사를 기준으로 필터링해 달라고 요청하는 경우, 먼저 name_search 도구를 사용하여 철자를 확인해야 합니다.
그렇지 않으면 데이터베이스의 테이블을 살펴보고 쿼리할 수 있는 항목을 확인할 수 있습니다.
그런 다음 가장 관련성이 높은 테이블의 스키마를 쿼리해야 합니다.
"""

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    extra_tools=custom_tool_list,
    suffix=custom_suffix,
)

Let's try it out:

In [27]:
agent.run("alice in chains는 몇 개의 앨범을 가지고 있나요?")



> Entering new AgentExecutor chain...

Invoking: `name_search` with `{'query': 'alice in chains'}`


Alice In Chains

Metallica

Pearl Jam

Pearl Jam
Invoking: `sql_db_list_tables` with ``


albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracks
Invoking: `sql_db_schema` with `{'table_names': 'albums, artists'}`



CREATE TABLE albums (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES artists ("ArtistId")
)

/*
3 rows from albums table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE artists (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from artists table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/
Invoking: `sql_db_query` with `SELECT COUNT(*) FROM albums WHERE ArtistId = (SELECT Art

'Alice In Chains는 1개의 앨범을 가지고 있습니다.'

보시다시피 에이전트는 이 특정 아티스트에 대한 데이터베이스를 올바르게 쿼리하는 방법을 확인하기 위해 `name_search` 도구를 사용했습니다.